In [1]:
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
import random

c:\Dissertation\dissertation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Dissertation\dissertation\lib\site-packages\torch\utils\tensorboard\__init__.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  tensorboard.__version__


In [2]:
def allWorldStages():
    worlds = [1, 2, 3, 4, 5, 6, 7, 8]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [1, 2, 3]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

env_names = [name for name in allWorldStages()]
env_names[0]

'SuperMarioBros-1-1-v0'

In [29]:
env_names

['SuperMarioBros-1-1-v0',
 'SuperMarioBros-1-2-v0',
 'SuperMarioBros-1-3-v0',
 'SuperMarioBros-2-1-v0',
 'SuperMarioBros-2-2-v0',
 'SuperMarioBros-2-3-v0',
 'SuperMarioBros-3-1-v0',
 'SuperMarioBros-3-2-v0',
 'SuperMarioBros-3-3-v0',
 'SuperMarioBros-4-1-v0',
 'SuperMarioBros-4-2-v0',
 'SuperMarioBros-4-3-v0',
 'SuperMarioBros-5-1-v0',
 'SuperMarioBros-5-2-v0',
 'SuperMarioBros-5-3-v0',
 'SuperMarioBros-6-1-v0',
 'SuperMarioBros-6-2-v0',
 'SuperMarioBros-6-3-v0',
 'SuperMarioBros-7-1-v0',
 'SuperMarioBros-7-2-v0',
 'SuperMarioBros-7-3-v0',
 'SuperMarioBros-8-1-v0',
 'SuperMarioBros-8-2-v0',
 'SuperMarioBros-8-3-v0']

In [3]:
def make_env(env_name):
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env,4,channels_order='last')
    return env

In [4]:
env = make_env(env_names[0])

c:\Dissertation\dissertation\lib\site-packages\gym\envs\registration.py:594: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
c:\Dissertation\dissertation\lib\site-packages\gym\core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
c:\Dissertation\dissertation\lib\site-packages\gym\wrappers\step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This wil

In [5]:
state = env.reset()
print(state.shape)

(1, 240, 256, 4)


In [6]:
#Import Os for file path management
import os

#Import PPO for algos
from stable_baselines3 import PPO
from stable_baselines3 import DQN

#Import base callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [7]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True
    
CHECKPOINT_DIR = './train'
LOG_DIR = './logs'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [9]:
#Create model with first env
model = DQN('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate= 0.000001, buffer_size=10000, batch_size=32)

Using cpu device
Wrapping the env in a VecTransposeImage.


c:\Dissertation\dissertation\lib\site-packages\stable_baselines3\common\buffers.py:220: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 4.92GB > 1.30GB
  "This system does not have apparently enough memory to store the complete "


In [ ]:
for env_name in env_names:
    env = make_env(env_name)
    model.set_env(env)
    model.learn(total_timesteps=10000, callback=callback)
    env.close()

In [10]:
model.load('./train/best_model_300000')

ValueError: unsupported pickle protocol: 5